### Importing Raw Data CSV file

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("M:/Data Science/Data Analyst/Github & LinkedIN Portfolio Projects/Manufacturing and Supply Chain Analytics/raw_manufacturing_data.csv")
print("Original data shape:", df.shape)
print(df.head())

Original data shape: (963, 16)
       Plant_ID Production_Date Shift  Planned_Units  Units_Produced  \
0  R_Aurangabad      23-02-2024     A            228             188   
1      R_Chakan      17-09-2024     C            190             181   
2        R_Pune      13-09-2024     B            197             182   
3  R_Aurangabad      11-04-2024     A            212             193   
4      R_Chakan      19-10-2024     A            197             159   

   Defective_Units Defect_Type  Supplier_ID  Raw_Material  Lead_Time_Days  \
0               14     Welding           ZF  Engine_Parts              11   
1                4    Painting        Valeo   Electronics              15   
2               13    Assembly  Continental   Electronics               5   
3               13    Assembly  Continental   Electronics               8   
4                0    Painting           ZF    Body_Parts               6   

   Downtime_Minutes    Downtime_Reason  Inventory_Level  Reorder_Level  \

### Convert date

In [2]:
df['Production_Date'] = pd.to_datetime(df['Production_Date'])
df['Production_Date'] = df['Production_Date'].dt.strftime('%d-%m-%Y')
print("After date conversion:")
print(df['Production_Date'].head())

After date conversion:
0    23-02-2024
1    17-09-2024
2    13-09-2024
3    11-04-2024
4    19-10-2024
Name: Production_Date, dtype: object


C:\Users\admin\AppData\Local\Temp\ipykernel_7272\668658733.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Production_Date'] = pd.to_datetime(df['Production_Date'])


### Remove negative or illogical values

In [3]:
print("Before removing negative values:")
print(f"Units_Produced < 0: {(df['Units_Produced'] < 0).sum()} rows")
print(f"Defective_Units < 0: {(df['Defective_Units'] < 0).sum()} rows")
df = df[df['Units_Produced'] >= 0]
df = df[df['Defective_Units'] >= 0]

print("\nAfter removing negative values:")
print(f"Units_Produced < 0: {(df['Units_Produced'] < 0).sum()} rows")
print(f"Defective_Units < 0: {(df['Defective_Units'] < 0).sum()} rows")
print(f"New data shape: {df.shape}")

Before removing negative values:
Units_Produced < 0: 0 rows
Defective_Units < 0: 0 rows

After removing negative values:
Units_Produced < 0: 0 rows
Defective_Units < 0: 0 rows
New data shape: (963, 16)


### Cap defects

In [4]:
print("Before capping defects:")
print(f"Rows where Defective_Units > Units_Produced: {(df['Defective_Units'] > df['Units_Produced']).sum()}")

df['Defective_Units'] = np.where(
    df['Defective_Units'] > df['Units_Produced'],
    df['Units_Produced'],
    df['Defective_Units']
)

print("\nAfter capping defects:")
print(f"Rows where Defective_Units > Units_Produced: {(df['Defective_Units'] > df['Units_Produced']).sum()}")

Before capping defects:
Rows where Defective_Units > Units_Produced: 0

After capping defects:
Rows where Defective_Units > Units_Produced: 0


### KPIs

In [5]:
df['Defect_Rate'] = round((df['Defective_Units'] / df['Units_Produced']) * 100, 2)
df['Production_Efficiency'] = round((df['Units_Produced'] / df['Planned_Units']) * 100, 2)
df['Inventory_Status'] = np.where(
    df['Inventory_Level'] < df['Reorder_Level'], 'Below Reorder', 'Optimal'
)

print("After creating KPIs:")
print(df[['Units_Produced', 'Defective_Units', 'Defect_Rate', 'Production_Efficiency', 'Inventory_Status']].head())

After creating KPIs:
   Units_Produced  Defective_Units  Defect_Rate  Production_Efficiency  \
0             188               14         7.45                  82.46   
1             181                4         2.21                  95.26   
2             182               13         7.14                  92.39   
3             193               13         6.74                  91.04   
4             159                0         0.00                  80.71   

  Inventory_Status  
0    Below Reorder  
1          Optimal  
2          Optimal  
3    Below Reorder  
4          Optimal  


### Handle missing defects

In [6]:
print("Before handling missing defects:")
print(df['Defect_Type'].value_counts(dropna=False).head())

df['Defect_Type'] = df['Defect_Type'].replace('None', np.nan)
df['Defect_Type'] = df['Defect_Type'].fillna('No Defect')

print("\nAfter handling missing defects:")
print(df['Defect_Type'].value_counts().head())

Before handling missing defects:
Defect_Type
Welding       199
Painting      199
Assembly      195
Electrical    186
NaN           184
Name: count, dtype: int64

After handling missing defects:
Defect_Type
Welding       199
Painting      199
Assembly      195
Electrical    186
No Defect     184
Name: count, dtype: int64


### Display final data summary

In [7]:
print("Final data summary:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nData types:")
print(df.dtypes)
print("\nMissing values:")
print(df.isnull().sum())
print("\nSample of final cleaned data:")
print(df.head())

Final data summary:
Shape: (963, 19)
Columns: ['Plant_ID', 'Production_Date', 'Shift', 'Planned_Units', 'Units_Produced', 'Defective_Units', 'Defect_Type', 'Supplier_ID', 'Raw_Material', 'Lead_Time_Days', 'Downtime_Minutes', 'Downtime_Reason', 'Inventory_Level', 'Reorder_Level', 'Production_Cost_Per_Unit', 'Delivery_Delay_Days', 'Defect_Rate', 'Production_Efficiency', 'Inventory_Status']

Data types:
Plant_ID                     object
Production_Date              object
Shift                        object
Planned_Units                 int64
Units_Produced                int64
Defective_Units               int64
Defect_Type                  object
Supplier_ID                  object
Raw_Material                 object
Lead_Time_Days                int64
Downtime_Minutes              int64
Downtime_Reason              object
Inventory_Level               int64
Reorder_Level                 int64
Production_Cost_Per_Unit    float64
Delivery_Delay_Days           int64
Defect_Rate         

### Saving the cleaned data

In [8]:
df.to_csv("M:/Data Science/Data Analyst/Github & LinkedIN Portfolio Projects/Manufacturing and Supply Chain Analytics/clean_manufacturing_data.csv", index=False)
print("\nCleaned data saved successfully!")


Cleaned data saved successfully!


### Exporting Cleaned Data To PostgreSQL

In [12]:
import pandas as pd
from sqlalchemy import create_engine

df = pd.read_csv("M:/Data Science/Data Analyst/Github & LinkedIN Portfolio Projects/Manufacturing and Supply Chain Analytics/clean_manufacturing_data.csv")

DB_USER = "postgres"
DB_PASSWORD = 987021
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "Manufacturing_&_SC_analytics"


engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

df.to_sql(
    name="R_manufacturing",
    con=engine,
    schema="public",
    if_exists="append",  
    index=False,
    method="multi",
    chunksize=1000
)

print("Data successfully exported to PostgreSQL.")

Data successfully exported to PostgreSQL.
